# Проект для «Викишоп» Lemma Spacy

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import spacy
from tqdm import notebook
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
import catboost
from catboost import CatBoostClassifier
import lightgbm as lgb

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavelbekirov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
try:
    df = pd.read_csv('toxic_comments.csv')
    print('датасет загружен локально')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
    print('датасет загружен с сервера')

датасет загружен локально


In [3]:
df[:3]

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


In [4]:
df['text'][1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [5]:
def clear_text(text):
    text = re.sub(r'[^A-Za-z]', ' ', text)
    return " ".join(text.split())

In [6]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
df['lemma'] = ''
for i in notebook.tqdm(range(0, len(df))):
    doc = nlp(df['text'][i])
    df['lemma'][i] = clear_text(" ".join([token.lemma_ for token in doc]))
df

  0%|          | 0/159292 [00:00<?, ?it/s]

/var/folders/n5/_dbcy3_n789_1_zt9g95px240000gn/T/ipykernel_75696/1094484336.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemma'][i] = clear_text(" ".join([token.lemma_ for token in doc]))


,Unnamed: 0,text,toxic,lemma
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,D aww he match this background colour I be see...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more I can not make any real suggestion on imp...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that be a ho...
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there s no actual article for pros...
159290,159449,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


In [7]:
stop_words = set(stopwords.words('english'))
corpus = df['lemma'].values
count_vect = CountVectorizer(stop_words = stop_words)
bow = count_vect.fit_transform(corpus)

In [8]:
bow.shape

(159292, 157552)

In [9]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf = count_tf_idf.fit_transform(corpus)

In [10]:
# выборки
features = tf_idf
target = df['toxic']

In [11]:
# размеры выборок
target.shape, features.shape

((159292,), (159292, 157552))

In [12]:
# сплит
features_train, features_test, target_train, target_test = train_test_split \
    (features, target, test_size = 0.3, random_state = 12345)

In [13]:
# # Линейная логистическая модель и метрика
# model_reg = CatBoostClassifier()
# model_reg.fit(features_train, target_train)
# predicts = model_reg.predict(features_train)
# predicts_test = model_reg.predict(features_test)
# print(f1_score(predicts, target_train))
# print(f1_score(predicts_test, target_test))

In [ ]:
model_reg = lgb.LGBMClassifier()
model_reg.fit(features_train, target_train)
predicts = model_reg.predict(features_train)
predicts_test = model_reg.predict(features_test)
print(f1_score(predicts, target_train))
print(f1_score(predicts_test, target_test))

In [1]:
print('gg')

gg


## Выводы
Подобрали модель Бертом и логистической регрессией которая дает результат 0.92 на тесте.
Мы использовали маленькую часть выборки для ускорения процесса, но использую хорошие мощности
можно перебрать и все объекты.

## Чек-лист проверки

    - [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны